In [1]:
%pip install bs4


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install lxml


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests

url = "https://maths.iiserb.ac.in/faculty"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers, verify=False)
html = response.text

print(html[:500])

<!DOCTYPE html>
<html>
<p style="margin-bottom: 5px;  padding-right: 29px;  padding-left: 15px;  margin-top: 6px;  margin-left: 21px;  color: white;"></p>
<head>
<title>Department of Mathematics </title>
<meta charset="UTF-8">
<meta http-equiv="Content-Security-Policy: connect-src 'self';
                         font-src 'self';
						 script-src 'self';
                         frame-src 'self';
                         img-src 'self';
                         manifest-src 'self';



/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maths.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [8]:
import requests
from bs4 import BeautifulSoup
import csv
import time

BASE_URL = "https://maths.iiserb.ac.in"
FACULTY_PAGE = f"{BASE_URL}/faculty"

headers = {"User-Agent": "Mozilla/5.0"}

# Fetch the main faculty listing page
response = requests.get(FACULTY_PAGE, headers=headers, verify=False)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

# Select each faculty block
faculty_blocks = soup.select("li.one_quarter, li.one_quarter.first")

with open("iiserbmath_faculty_phd_postdoc.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Name", "Designation", "Research Area", "Email", "Phone", "Profile Link", "PhD", "Postdoc"
    ])

    for block in faculty_blocks:
        # --- Basic details from the main page ---
        name_tag = block.select_one("figcaption a[href*='faculty_details2']")
        name = name_tag.text.strip() if name_tag else ""
        rel_link = name_tag.get("href") if name_tag else ""
        profile_link = rel_link if rel_link.startswith("http") else f"{BASE_URL}/{rel_link}" if rel_link else ""

        figcaption = block.find("figcaption")
        spans = figcaption.find_all("span", style=lambda x: x and "font-size" in x) if figcaption else []
        designation = spans[0].text.strip() if len(spans) > 0 else ""
        research_area = spans[1].text.strip() if len(spans) > 1 else ""

        email_tag = figcaption.select_one("a[href*='@iiserb.ac.in']") if figcaption else None
        email = email_tag.text.strip() if email_tag else ""

        phone_span = figcaption.find("span", string=lambda x: x and "+91" in x) if figcaption else None
        phone = phone_span.text.strip() if phone_span else ""

        # --- Extract Ph.D. and Postdoc info from profile page ---
        phd = ""
        postdoc = ""

        if profile_link:
            try:
                prof_resp = requests.get(profile_link, headers=headers, verify=False)
                prof_resp.raise_for_status()
                prof_soup = BeautifulSoup(prof_resp.text, "html.parser")

                # Find the "Academic Details" section (orange accordion)
                acad_section = prof_soup.select_one("div.accordion-wrapper:has(.accordion-title.orange)")
                if acad_section:
                    for li in acad_section.select("ul.list li"):
                        text = li.get_text(strip=True)
                        if not text:
                            continue
                        if "Ph.D" in text or "PhD" in text:
                            phd = text
                        elif "Postdoc" in text or "Postdoctoral" in text:
                            postdoc = text

                time.sleep(0.3)  # polite delay between requests

            except Exception as e:
                print(f"⚠️ Failed to fetch {profile_link}: {e}")

        writer.writerow([name, designation, research_area, email, phone, profile_link, phd, postdoc])

print("✅ Scraping completed! Data saved to iiserbmath_faculty_phd_postdoc.csv")


/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maths.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maths.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maths.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/codespace/.local

✅ Scraping completed! Data saved to iiserbmath_faculty_phd_postdoc.csv
